In [20]:
# Import dependancies
import json
import random
from citipy import citipy
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np

# Import API key
from config import weather_api_key

In [36]:
city_df = pd.DataFrame({"Lat":np.random.uniform(-90,90,1300),"Lng":np.random.uniform(-180,180,1300)})
city_df.head()

,Lat,Lng
0,-86.376034,109.200709
1,-67.105091,169.535569
2,69.273948,101.563753
3,-34.566562,-39.449126
4,-60.155919,76.908600


In [37]:
#  Generate Cities List
cities_name = []
for x in range(0,len(city_df)):
    lat = city_df.iloc[x]["Lat"]
    lng = city_df.iloc[x]["Lng"]
    city = citipy.nearest_city(lat,lng)
    name = city.city_name
    cities_name.append(name)

city_df["City"] = cities_name
city_df.head()

,Lat,Lng,City
0,-86.376034,109.200709,albany
1,-67.105091,169.535569,bluff
2,69.273948,101.563753,khatanga
3,-34.566562,-39.449126,laguna
4,-60.155919,76.908600,souillac


In [39]:
no_dup_cities = city_df.drop_duplicates("City", keep="first")
no_dup_cities


,Lat,Lng,City
0,-86.376034,109.200709,albany
1,-67.105091,169.535569,bluff
2,69.273948,101.563753,khatanga
3,-34.566562,-39.449126,laguna
4,-60.155919,76.908600,souillac
...,...,...,...
1283,36.498433,-84.269511,clinton
1288,-17.360301,-142.604660,tautira
1291,14.613470,62.897670,sur
1295,38.305023,-71.450656,mastic beach


In [54]:
url = "http://api.openweathermap.org/data/2.5/weather?q=London&appid=77eb7fbda02ce8516e38146569387265"
response = requests.get(url).json()
response

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 300.11,
  'feels_like': 300.91,
  'temp_min': 297.25,
  'temp_max': 302.14,
  'pressure': 1021,
  'humidity': 56},
 'visibility': 10000,
 'wind': {'speed': 2.06, 'deg': 150},
 'clouds': {'all': 0},
 'dt': 1626775958,
 'sys': {'type': 2,
  'id': 268730,
  'country': 'GB',
  'sunrise': 1626754030,
  'sunset': 1626811571},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [55]:
#  Perform API Calls
params = {"appid": weather_api_key,
        "units": "metric",
        }
base_url = "http://api.openweathermap.org/data/2.5/weather?"
for index, row in no_dup_cities.iterrows():
    city_name = row["City"]

    params["q="] = city_name
    print(f"Processing Record {index} of set {city_name}")
    results = requests.get(base_url, params=params).json()
    
    try:
        no_dup_cities.loc[index, "temp"] = results["main"]["temp_max"]
        no_dup_cities.loc[index, "humidity"] = results["main"]["humidity"]
        no_dup_cities.loc[index, "cloud cover"] = results["clouds"]["all"]
        no_dup_cities.loc[index, "wind speed"] = results["wind"]["speed"]

    except(KeyError, IndexError):
        print("Missing data ...... Skipping")
# Perform a weather check on each city using a series of successive API calls.
# Include a print log of each city as it'sbeing processed (with the city number and city name).

Processing Record 0 of set albany
Missing data ...... Skipping
Processing Record 1 of set bluff
Missing data ...... Skipping
Processing Record 2 of set khatanga
Missing data ...... Skipping
Processing Record 3 of set laguna
Missing data ...... Skipping
Processing Record 4 of set souillac
Missing data ...... Skipping
Processing Record 5 of set attawapiskat
Missing data ...... Skipping
Processing Record 6 of set ushuaia
Missing data ...... Skipping
Processing Record 7 of set victoria
Missing data ...... Skipping
Processing Record 8 of set atuona
Missing data ...... Skipping
Processing Record 9 of set goderich
Missing data ...... Skipping
Processing Record 10 of set saldanha
Missing data ...... Skipping
Processing Record 11 of set vaini
Missing data ...... Skipping
Processing Record 12 of set dikson
Missing data ...... Skipping
Processing Record 13 of set kodiak
Missing data ...... Skipping
Processing Record 14 of set moscow
Missing data ...... Skipping
Processing Record 15 of set avarua


In [56]:
no_dup_cities.head()

,Lat,Lng,City
0,-86.376034,109.200709,albany
1,-67.105091,169.535569,bluff
2,69.273948,101.563753,khatanga
3,-34.566562,-39.449126,laguna
4,-60.155919,76.908600,souillac


In [ ]:
#  Convert Raw Data to DataFrame
# Export the city data into a .csv.
# Display the DataFrame

In [ ]:
#  Inspect the data and remove the cities where the humidity > 100%.
# Skip this step if there are no cities that have humidity > 100%.

In [ ]:
#  Get the indices of cities that have humidity over 100%.

In [ ]:
 # Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

Plotting the data

In [ ]:
# Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.Save the plotted figures as .pngs.

In [ ]:
# Latitude vs. Temperature Plot

In [ ]:
#  Latitude vs. Humidity Plot

In [ ]:
#  Latitude vs. Cloudiness Plot

In [ ]:
# Latitude vs. Wind Speed Plot

Linear Regression

In [ ]:
#  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression